# Содержание:

    1) Метод doc2vec с классификаторами:
            1) Ridge Classifier
            2) Perceptron
            3) Passive Agressive
            4) BeurnolliNB
            5) MultinomialNB
            6) KNeighbors
            7) RandomForest
            8) NearestCentroid
            9) SGDClassifier
            10) LinearSVC

In [2]:
%matplotlib inline
import os
from lxml import etree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# License: BSD 3 clause

from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import pymorphy2
import gensim
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [3]:
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
       self.labels_list = labels_list
       self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield gensim.models.doc2vec.LabeledSentence(words=doc.split(),tags=[self.labels_list[idx]])

## Загружаю данные:

In [3]:
sphere_6 = defaultdict(int)
genre_fi_7 = defaultdict(int)
type_8 = defaultdict(int)
topic_9 = defaultdict(int)

i = 0
f = open('./out.csv', 'r')
for line in f:
    i += 1
    attrs = line.split(';')
    sphere_6[attrs[6]] += 1
    genre_fi_7[attrs[7]] += 1
    type_8[attrs[8]] += 1
    topic_9[attrs[9]] += 1

In [82]:
# type_8

In [80]:
# cl.keys()

In [4]:
from Levenshtein import distance

In [5]:
f = open('./out.csv', 'r')
file_hash_all = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    file_name = os.path.basename(path)
    dir_name = os.path.dirname(path)
    full_dir = os.path.join('/Users/Valeriya/Desktop/CoursePaper/corpus', dir_name)
#     full_dir = os.path.join('./corpus', dir_name)
    files = os.listdir(full_dir)
    if file_name in files:
        file_hash_all[file_name] = class_name
    elif file_name + '.xml' in files:
        file_hash_all[file_name + '.xml'] = class_name
    elif file_name + '.xhtml' in files:
        file_hash_all[file_name + '.xhtml'] = class_name
    else:
        edit = 1000
        true_name = ''
        for file_in_dir in files:
            dist = distance(file_name, file_in_dir)
            if dist < edit:
                edit = dist
                true_name = file_in_dir
        file_hash_all[true_name] = class_name

In [6]:
len(file_hash_all)

14382

In [114]:
f = open('./out.csv', 'r')
cl = {'отчет':0,'поздравление':0, 'интервью':0, 'репортаж':0, 'хроника':0, 
      'объявление':0, 'комментарий':0, 'совет':0, 'анонс':0, 'заметка':0, 'информационное':0, 'статья':0}
# cl = {'поздравление':0, 'интервью':0,  
#       'объявление':0, 'совет':0, 'информационное':0, 'статья':0}
file_hash = {}
for file_name, class_name in file_hash_all.items():
    if class_name in cl:
        cl[class_name] +=1
        file_hash[file_name] = class_name

In [115]:
len(file_hash)

10880

In [116]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    pos = p.tag.POS
    if pos:
        if 'ADJ' in pos:
            pos = 'ADJ'
    else:
        pos = ''
    return '{}_{}'.format(
        p.normal_form,
        pos
    )

2017-05-19 22:08:15,456 INFO Loading dictionaries from /Users/Valeriya/anaconda/lib/python3.5/site-packages/pymorphy2_dicts/data
2017-05-19 22:08:15,812 INFO format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [30]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text, open_name

In [27]:
# sent = []
# for tree, class_text, open_name in read_txt('./corpus'):
#     all_p = tree.xpath('//p')
#     joined = '\n'.join([p.text for p in all_p if p.text is not None])
#     sent.append((joined, class_text))

In [288]:
# words = sum([len(text[0].split()) for text in sent])
# words

In [117]:
texts = []
for tree, class_text, open_name in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
# for tree, class_text, open_name in read_txt('./corpus'):
    body = tree.xpath('//body')
    text = '\n'.join([elem.text for elem in body[0] if elem.text])
#     if class_text != 'поздравление':
#         class_text = 'другие'
    if text:
        texts.append((text, class_text, open_name))
    else:
        print(open_name)

/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/center/vest/390676.xhtml
/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451400.xhtml
/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451403.xhtml
/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451404.xhtml


In [30]:
# print(texts[0])

In [291]:
# words = sum([len(text[0].split()) for text in sent])
# words

In [118]:
len_texts = len(texts)

In [119]:
len_texts

10877

In [106]:
# m = 'ruwikiruscorpora_0_300_20.bin.gz'
# model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

In [120]:
only_texts = [text[0] for text in texts]
classes = [text[1] for text in texts]

In [121]:
docLabels = [str(i) for i in range(len_texts)]

it = DocIterator(only_texts, docLabels)

model = gensim.models.Doc2Vec(window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it)
for epoch in range(10):
    model.train(it)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    model.train(it)

2017-05-19 22:08:48,357 INFO collecting all words and their counts
2017-05-19 22:08:48,360 INFO PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-19 22:08:51,492 INFO PROGRESS: at example #10000, processed 5030464 words (1606582/s), 513470 word types, 10000 tags
2017-05-19 22:08:51,795 INFO collected 541230 word types and 10877 unique tags from a corpus of 10877 examples and 5460068 words
2017-05-19 22:08:51,796 INFO Loading a fresh vocabulary
2017-05-19 22:08:53,024 INFO min_count=5 retains 92803 unique words (17% of original 541230, drops 448427)
2017-05-19 22:08:53,026 INFO min_count=5 leaves 4803883 word corpus (87% of original 5460068, drops 656185)
2017-05-19 22:08:53,460 INFO deleting the raw counts dictionary of 541230 items
2017-05-19 22:08:53,513 INFO sample=0.001 downsamples 21 most-common words
2017-05-19 22:08:53,515 INFO downsampling leaves estimated 4259038 word corpus (88.7% of prior 4803883)
2017-05-19 22:08:53,517 INFO estimated required m

In [122]:
len(model.docvecs)

10877

In [123]:
features = {}
# with open('formal_features_3.tsv', 'r') as f:
with open('formal_features_3.tsv', 'r') as f:
    for line in f:
        items = line.rstrip().split('\t')
        path = items[0]
        feats = items[1::]
        features[path] = feats

In [124]:
data = []
for i, (text, class_name, open_name) in enumerate(texts):
    text_vectors = []
    words = text.split()
    doc2vec_vector = list(model.docvecs[i])
    open_name = open_name.replace('/Users/Valeriya/Desktop/CoursePaper/', './')
    formal_features = features[open_name][::]
    n_excl = 0
    for item in text[0]:
        if item == '!':
            n_excl+=1

    n_ques = 0
    for item in text[0]:
        if item == '?':
            n_ques+=1
    formal_features.append(n_excl)
    formal_features.append(n_ques)
    
    formal_features = [float(x) for x in formal_features]
    formal_features = formal_features[:2:] + [x/formal_features[0] for x in formal_features[2::]]
    
    data.append(doc2vec_vector+formal_features)


# data = pd.DataFrame(texts_norm)
# data['class'] = classes

In [125]:
len(data)

10877

In [126]:
X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.3)

In [18]:
min_max_scaler = MinMaxScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

## Классификаторы

In [128]:
from sklearn.metrics import classification_report, confusion_matrix

clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)
print(classification_report(y_test, y_pred1))
print(confusion_matrix(y_test, y_pred1))
# clf2 = LogisticRegression()
# clf2.fit(X_train, y_train)
# y_pred2 = clf2.predict(X_test)
# print(classification_report(y_test, y_pred2))
# print(confusion_matrix(y_test, y_pred2))
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf1, X_train, y_train, cv=5)
print(scores)
# results = [[x[i] for x in results] for i in range(2)]

/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                precision    recall  f1-score   support

         анонс       0.67      0.06      0.11        70
       заметка       0.58      0.93      0.72      1132
      интервью       0.69      0.22      0.33       261
информационное       0.51      0.37      0.43       315
   комментарий       0.75      0.01      0.03       202
    объявление       0.64      0.31      0.42       126
         отчет       0.75      0.03      0.07       173
  поздравление       0.94      0.58      0.72        55
      репортаж       0.00      0.00      0.00        95
         совет       1.00      0.03      0.05        36
        статья       0.63      0.90      0.74       721
       хроника       0.50      0.01      0.03        78

   avg / total       0.61      0.60      0.52      3264

[[   4   37    0   18    0   11    0    0    0    0    0    0]
 [   1 1056    0   38    0    3    0    0    0    0   34    0]
 [   0   66   57    0    1    0    0    0    0    0  137    0]
 [   1  184    0  118   

In [ ]:
#с дефолтом было 624
# 100 - 6435